결측 값 행 제거 + 결측 값 많은 열 제거 + 수치 데이터 정규화 작업(수치 데이터이지만 값이 정해진 경우에는 label encoder로 변환)<br/>
음수 WTI는 새로운 컬럼으로 만들어 처리<br/>
10 단위 1로 수정

In [1]:
import warnings
warnings.filterwarnings(action="ignore")
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [4]:
train_df = pd.read_parquet('../data/train.parquet').drop(columns=['SAMPLE_ID'])
test_df = pd.read_parquet('../data/test.parquet').drop(columns=['SAMPLE_ID'])

In [5]:
train_df.isna().sum()

ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ATA                        0
ID                         0
BREADTH                    1
BUILT                      0
DEADWEIGHT                 0
DEPTH                      1
DRAUGHT                    1
GT                         0
LENGTH                     1
SHIPMANAGER                0
FLAG                       0
U_WIND                153486
V_WIND                153486
AIR_TEMPERATURE       154365
BN                    153486
ATA_LT                     0
DUBAI                      0
BRENT                      0
WTI                        0
BDI_ADJ                    0
PORT_SIZE                  0
CI_HOUR                    0
dtype: int64

In [6]:
# datetime 컬럼 처리
train_df['ATA'] = pd.to_datetime(train_df['ATA'])
test_df['ATA'] = pd.to_datetime(test_df['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train_df, test_df]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour
    df['minute'] = df['ATA'].dt.minute
    df['weekday'] = df['ATA'].dt.weekday

# # datetime 컬럼 제거
train_df.drop(columns=['ATA', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN'], inplace=True)
test_df.drop(columns=['ATA', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BN'], inplace=True)

train_df.dropna(inplace=True)

In [7]:
train_df.iloc[0]

ARI_CO                                         CN
ARI_PO                                       EKP8
SHIP_TYPE_CATEGORY                           Bulk
DIST                                    30.736578
ID                                        Z517571
BREADTH                                      30.0
BUILT                                          28
DEADWEIGHT                                  73100
DEPTH                                        20.0
DRAUGHT                                      10.0
GT                                          38600
LENGTH                                      220.0
SHIPMANAGER                                CHMT36
FLAG                  China, People's Republic Of
ATA_LT                                         12
DUBAI                                       42.01
BRENT                                       43.16
WTI                                         40.96
BDI_ADJ                                1407.66833
PORT_SIZE                                 0.00166


In [8]:
# WTI 처리
train_df["WIT<0"] = [0 if wti > 0 else 1 for wti in train_df["WTI"]]
train_df["WTI"] = [wti if wti > 0 else 0 for wti in train_df["WTI"]]

test_df["WIT<0"] = [0 if wti > 0 else 1 for wti in test_df["WTI"]]
test_df["WTI"] = [wti if wti > 0 else 0 for wti in test_df["WTI"]]

In [10]:
scaler = StandardScaler()
x = train_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH", "DUBAI", "BRENT", "WTI", "BDI_ADJ", "PORT_SIZE"]]
scaler.fit(x)
train_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH", "DUBAI", "BRENT", "WTI", "BDI_ADJ", "PORT_SIZE"]] = scaler.transform(x)
test_feautre = test_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH", "DUBAI", "BRENT", "WTI", "BDI_ADJ", "PORT_SIZE"]]
test_df[["DIST", "BUILT", "DEADWEIGHT", "GT", "LENGTH", "DUBAI", "BRENT", "WTI", "BDI_ADJ", "PORT_SIZE"]] = scaler.transform(test_feautre)

In [11]:
train_df.head()

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,...,BDI_ADJ,PORT_SIZE,CI_HOUR,year,month,day,hour,minute,weekday,WIT<0
0,CN,EKP8,Bulk,0.369560,Z517571,30.0,1.882568,0.149641,20.0,10.0,...,0.080405,0.935023,3.048333,2020,10,15,4,3,3,0
1,CN,EUC8,Container,1.420706,U467618,30.0,0.036071,-0.404179,20.0,10.0,...,1.002482,0.880050,17.138611,2019,9,17,2,55,1,0
2,CN,NGG6,Container,2.301099,V378315,50.0,-1.100235,0.808875,20.0,10.0,...,-1.008252,1.033596,98.827500,2019,2,23,6,43,5,0
3,JP,TMR7,Cargo,-0.625047,B726632,10.0,2.592759,-0.977036,10.0,0.0,...,-0.241418,-0.963324,0.000000,2020,9,18,22,6,4,0
4,RU,NNC2,Container,-0.339843,D215135,30.0,-0.674120,-0.566234,10.0,10.0,...,-0.325196,-0.811344,96.030556,2022,8,13,12,57,5,0


In [14]:
for name in ["BREADTH", "DEPTH", "DRAUGHT"]:
    train_df[name] = train_df[name].apply(lambda x: x // 10)
    test_df[name] = test_df[name].apply(lambda x: x // 10)

In [15]:
train_df.head()

,ARI_CO,ARI_PO,SHIP_TYPE_CATEGORY,DIST,ID,BREADTH,BUILT,DEADWEIGHT,DEPTH,DRAUGHT,...,BDI_ADJ,PORT_SIZE,CI_HOUR,year,month,day,hour,minute,weekday,WIT<0
0,CN,EKP8,Bulk,0.369560,Z517571,3.0,1.882568,0.149641,2.0,1.0,...,0.080405,0.935023,3.048333,2020,10,15,4,3,3,0
1,CN,EUC8,Container,1.420706,U467618,3.0,0.036071,-0.404179,2.0,1.0,...,1.002482,0.880050,17.138611,2019,9,17,2,55,1,0
2,CN,NGG6,Container,2.301099,V378315,5.0,-1.100235,0.808875,2.0,1.0,...,-1.008252,1.033596,98.827500,2019,2,23,6,43,5,0
3,JP,TMR7,Cargo,-0.625047,B726632,1.0,2.592759,-0.977036,1.0,0.0,...,-0.241418,-0.963324,0.000000,2020,9,18,22,6,4,0
4,RU,NNC2,Container,-0.339843,D215135,3.0,-0.674120,-0.566234,1.0,1.0,...,-0.325196,-0.811344,96.030556,2022,8,13,12,57,5,0


In [16]:
train_df.to_parquet("../data/train_v0.parquet")
test_df.to_parquet("../data/test_v0.parquet")